In [2]:
import edhec_risk_kit as erk
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthBegin
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


START LAB 201

In [3]:
brka_d = pd.read_csv('data/brka_d_ret.csv', parse_dates=True, index_col=0)
brka_m = brka_d.resample('1M').apply(erk.cumulate)
brka_m.index -= MonthBegin(1)
fff = erk.get_df(filename='data/F-F_Research_Data_Factors_m.csv', start_period=None, end_period=None,
                 format='%Y%m', reqd_strategies=None)
rr_capm = erk.regress(brka_m, fff[['Mkt-RF', 'RF']], '1990', rfcol='RF')
rr_ffm = erk.regress(brka_m, fff, '1990', rfcol='RF')
rr_ffm_nint = erk.regress(brka_m, fff, '1990', rfcol='RF', intercept=False)
print(rr_ffm.summary())
print(rr_ffm_nint.summary())

                            OLS Regression Results                            
Dep. Variable:                   BRKA   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     53.29
Date:                Fri, 22 May 2020   Prob (F-statistic):           2.59e-28
Time:                        09:20:09   Log-Likelihood:                 567.01
No. Observations:                 348   AIC:                            -1126.
Df Residuals:                     344   BIC:                            -1111.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.7096      0.063     11.350      0.0

START LAB 202

In [4]:
ind = erk.get_df('data/ind30_m_vw_rets.csv', start_period='2000', format='%Y%m', mode='return', end_period=None, reqd_strategies=None)
mgr_r = 0.3*ind["Beer"] + .5*ind["Smoke"] + 0.2*np.random.normal(scale=0.15/(12**.5), size=ind.shape[0])
weights = erk.style_analyze(mgr_r, ind, start_period='2000')
style_coeff = erk.regress(mgr_r, ind, intercept=False, excess_mkt=False).params


PLOT FUNCTION

In [5]:
def plot_style(wts_coeff: pd.Series):
    wts_coeff.sort_values(ascending=False, inplace=True)
    wt_data = [go.Bar(x=wts_coeff.index, y=wts_coeff)]
    pyo.iplot(wt_data)

STYLE MATCHING PLOT USING QUADRATIC OPTIMIZER - MORE ACCURATE

In [6]:
plot_style(weights)

STYLE MATCHING PLOT USING REGRESSION COEFF

In [7]:
plot_style(style_coeff)

STYLE ANALYZE OF BRKA_M FOR LAST DECADE FROM 2009

In [8]:
wts_09 = erk.style_analyze(brka_m, ind, start_period='2009')
plot_style(wts_09)



PENDING WORK - DOWNLOAD ANY MUTUAL FUND HISTORICAL DATA FROM YF AND ANALYZE USING FFF




LAB 203 START




In [9]:
# TO_PERIOD ('m') IS THEN CONVERTED TO TIMESTAMP OBJECT WHILE PLOTTING
ind_ew = erk.get_df('data/ind30_m_ew_rets.csv', to_per=True)
ind_vw = erk.get_df('data/ind30_m_vw_rets.csv', to_per=True)
sharpe_ratio_df = pd.DataFrame({'VW': erk.risk_info(ind_vw['1945':], only_sharpe=True),
                               'EW': erk.risk_info(ind_ew['1945':], only_sharpe=True)})


PLOT VW AND EW SHARPE RATIO




In [10]:
sr_data = [go.Bar(x=sharpe_ratio_df.index,
                  y=sharpe_ratio_df[col]) for col in sharpe_ratio_df.columns]
pyo.iplot(sr_data)

% OF CASES WHERE SR(EW) > SR(VW)




In [11]:
(sharpe_ratio_df['EW'] > sharpe_ratio_df['VW']).sum()/sharpe_ratio_df.shape[0]

0.6333333333333333

ROLLING WINDOW SR CALC FOR VW AND EW MARKET PF




In [12]:
sharpe_ratio_rolling_df = pd.DataFrame({
    'VW': ind_vw.rolling(window='1825D').apply(erk.risk_info, kwargs={'only_sharpe': True, 'rf': 0.03}, raw=True).mean(axis=1)['1945':].dropna(),
    'EW': ind_ew.rolling(window='1825D').apply(erk.risk_info, kwargs={'only_sharpe': True, 'rf': 0.03}, raw=True).mean(axis=1)['1945':].dropna()
})


PLOT ROLLING SR




In [13]:
sr_r_data = [go.Scatter(x=sharpe_ratio_rolling_df.index.to_timestamp(),
                      y=sharpe_ratio_rolling_df[col],
                      name=col) for col in sharpe_ratio_df.columns]
pyo.iplot(sr_r_data)

LAB START 204
CREATE CUSTOM ROLLING WINDOW TO HANDLE MULTI DIMENSION DATAFRAME FOR COMPUTATION
ULT - TRYING TO GET ROLLING RETURNS ACCORDING TO SPECIFIED WEIGHTING SCHEMES


In [3]:
ind_vw_74 = erk.get_df('data/ind49_m_vw_rets.csv', to_per=True, start_period='1974')
ind_mkt_wts_74 = erk.plot_corr_mktret(ind_ret_filename='data/ind49_m_vw_rets.csv',
                                       n_firms_filename='data/ind49_m_nfirms.csv',
                                       size_filename='data/ind49_m_size.csv',
                                       start_period='1974',
                                       end_period=None,
                                       to_per=True,
                                       retrieve_mkt_cap_wts=True,
                                       format='%Y%m')

In [4]:
mw_pf = erk.bt_roll(ind_vw_74, weighting_scheme=erk.weight_cw, window=60, cap_wts=ind_mkt_wts_74)
ew_pf = erk.bt_roll(ind_vw_74, weighting_scheme=erk.weight_ew, window=60, cap_wts=ind_mkt_wts_74)
ewt_pf = erk.bt_roll(ind_vw_74, weighting_scheme=erk.weight_ew, window=60, cap_wts=ind_mkt_wts_74, max_cw_mult=5, microcap_threshold=0.005)
btr = pd.DataFrame({'EW': ew_pf,
                    'MW': mw_pf,
                    'EWT': ewt_pf,
                    }).dropna()

# btr_wealth_index = pd.DataFrame({'EW_wealth_index': erk.drawdown(btr['EW'], retrive_index=True, is1p=False),
#                                  'MW_wealth_index': erk.drawdown(btr['MW'], retrive_index=True, is1p=False),
#                                  'EWT_wealth_index': erk.drawdown(btr['EWT'], retrive_index=True, is1p=False),
#                                  })

PLOT INDEX AND DISPLAY RISK INFO


In [6]:
btr_wealth_index = pd.DataFrame({key+'_wealth_index': erk.drawdown(btr[key], retrive_index=True, is1p=False) for key in btr})
data=[go.Scatter(x=btr_wealth_index.index.to_timestamp(),
                 y=btr_wealth_index[str],
                 name=str) for str in btr_wealth_index.columns]
pyo.iplot(data)
risk_info = erk.risk_info(btr)

In [16]:
risk_info

START QUIZ COURSE 2 WEEK 1


In [17]:
ind_vw_91 = erk.get_df('data/ind49_m_vw_rets.csv', to_per=True, start_period='1991')
fff_91 = erk.get_df(filename='data/F-F_Research_Data_Factors_m.csv', start_period='1991', end_period=None,
                 format='%Y%m', reqd_strategies=None, to_per=True)
fff_91_capm = fff_91[['Mkt-RF', 'RF']]
rr_capm_beer_1991 = erk.regress(ind_vw_91['Beer'], fff_91_capm, '1991', rfcol='RF')
rr_capm_beer_2013 = erk.regress(ind_vw_91['Beer'], fff_91_capm, '2013', rfcol='RF')
rr_capm_steel_1991 = erk.regress(ind_vw_91['Steel'], fff_91_capm, '1991', rfcol='RF')
rr_capm_steel_2013 = erk.regress(ind_vw_91['Steel'], fff_91_capm, '2013', rfcol='RF')

In [18]:
print(rr_capm_steel_2013.summary())

                            OLS Regression Results                            
Dep. Variable:                  Steel   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     62.18
Date:                Tue, 19 May 2020   Prob (F-statistic):           2.97e-11
Time:                        18:43:00   Log-Likelihood:                 116.15
No. Observations:                  72   AIC:                            -228.3
Df Residuals:                      70   BIC:                            -223.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         1.4169      0.180      7.885      0.0

In [19]:
regress_results = erk.regress(ind_vw_91, fff_91_capm, '1991', '1993', rfcol='RF').params
regress_results.columns = ind_vw_91.columns
regress_results.loc['Mkt-RF'].sort_values(ascending=False).tail()

Oil      0.550604
Coal     0.529958
Util     0.379527
Mines    0.274762
Gold    -0.553289
Name: Mkt-RF, dtype: float64

In [20]:
regress_results_ff = erk.regress(ind_vw_91, fff_91, '1991', rfcol='RF').params
regress_results_ff.columns = ind_vw_91.columns
regress_results_ff.loc['SMB'].sort_values(ascending=False).tail()

Insur   -0.251649
Food    -0.267873
Smoke   -0.307794
Drugs   -0.319486
Beer    -0.358655
Name: SMB, dtype: float64

In [21]:
regress_results_ff = erk.regress(ind_vw_91, fff_91, '1991', rfcol='RF').params
regress_results_ff.columns = ind_vw_91.columns
regress_results_ff.loc['HML'].sort_values(ascending=False).tail()

Drugs   -0.228132
LabEq   -0.261102
Chips   -0.600139
Hardw   -0.690417
Softw   -0.852422
Name: HML, dtype: float64

START WEEK 2


